In [1]:
import tensorflow as tf
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
#path the video
vidcap = cv2.VideoCapture(r"D:\LBP-shortcut\data\xy.mp4")
success, image = vidcap.read()
count = 0
while vidcap.isOpened():
    success, image = vidcap.read()
    cv2.imwrite(r"D:\LBP-shortcut\data\Data\fire\%d.png" % count, image)
    count+=1

In [3]:
#path the video
# vidcap1 = cv2.VideoCapture(r"D:\LBP-shortcut\data\no-fire.mp4")
# success, image1 = vidcap1.read()
# count1 = 0
# while vidcap1.isOpened():
#     success, image1 = vidcap1.read()
#     cv2.imwrite(r"D:\LBP-shortcut\data\Data\no-fire\%d.png" % count1, image1)
#     count1+=1

In [61]:
batch_size=64

training_datagenarator= ImageDataGenerator(rescale=1./255,horizontal_flip=True,
    vertical_flip=True,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2,validation_split=0.1)

In [62]:
train=training_datagenarator.flow_from_directory(r'D:\LBP-shortcut\data\Data',
                                                target_size=(224, 224),color_mode='rgb',
                                       class_mode='binary', batch_size=batch_size,subset='training')

validation=training_datagenarator.flow_from_directory(r'D:\LBP-shortcut\data\Data',
                                                target_size=(224, 224),color_mode='rgb',
                                       class_mode='binary', batch_size=batch_size,subset='validation')

Found 1470 images belonging to 2 classes.
Found 163 images belonging to 2 classes.


In [63]:
cnn = tf.keras.models.Sequential()
# adding first layer
cnn.add(tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',input_shape=[224,224,3]))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

# adding second layer
cnn.add(tf.keras.layers.Conv2D(filters=64,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

# adding third layer
cnn.add(tf.keras.layers.Conv2D(filters=256,kernel_size=3,padding='same',activation='relu'))
cnn.add(tf.keras.layers.MaxPool2D(pool_size=2))

# Flattening
cnn.add(tf.keras.layers.Flatten())

# Fully connected layer
cnn.add(tf.keras.layers.Dense(units=128,activation='relu'))

# Output layers
cnn.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))

In [64]:
cnn.compile(optimizer='Adam',loss='binary_crossentropy',metrics=['accuracy'])

In [65]:
from tensorflow.keras.models import load_model
cnn=load_model(r'D:\LBP-shortcut\data\model\fire.h5')

In [66]:
from tensorflow.keras.preprocessing import image
import numpy as np
import matplotlib.pyplot as plt
import PIL
import imgaug as ia
from imgaug.augmentables.bbs import BoundingBox
import skvideo.io
import glob
from moviepy.editor import *

In [67]:
# image_for_testing='D:/LBP-shortcut/data/Data/fire'
i=0
Image = []
for i in range(count):

    image_for_testing='D:/LBP-shortcut/data/Data/fire'+"/"+str(i)+".png"

    
    img = cv2.imread(str(image_for_testing))

    # convert to grayscale
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    # threshold
    thresh = cv2.threshold(gray,120,255,cv2.THRESH_BINARY)[1]

    # get contours
    result = img.copy()
    contours = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    contours = contours[0] if len(contours) == 2 else contours[1]
    for cntr in contours:
        x,y,w,h = cv2.boundingRect(cntr)
        cv2.rectangle(result, (x, y), (x+w, y+h), (0, 0, 255), 2)

 
    # save resulting image
    cv2.imwrite('Fire-Images/fire'+str(i)+'.jpg',result)


    Image = Image + ['Fire-Images/fire'+str(i)+'.jpg']



clips = []
for i in range(len(Image)):
    clip =  ImageClip(Image[i]).set_duration(2)
    clips.append(clip)

video_clip = concatenate_videoclips(clips, method='compose')
video_clip = video_clip.speedx(40)
video_clip.write_videofile("paperFire.mp4", fps=60, remove_temp=True, codec="libx264", audio_codec="aac")


Moviepy - Building video paperFire.mp4.
Moviepy - Writing video paperFire.mp4



Moviepy - Done !
Moviepy - video ready paperFire.mp4
